In [2]:
import pandas as pd
df=pd.read_csv("datasetwg.csv")
df.shape

(9460980, 43)

In [3]:
# 💾 Script LightGBM para predecir tn en febrero 2020

import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

# 1. Cargar datos
df = pd.read_csv("datasetwg.csv")
df['periodo'] = pd.to_datetime(df['periodo'])

# 2. Agregar ventas por producto y mes
df_prod = df.groupby(['product_id', 'periodo'])['tn'].sum().reset_index()

# 3. Crear lags
def crear_lags(df, lags=[1, 2, 3]):
    df_lags = df.copy()
    for lag in lags:
        df_lags[f'lag_{lag}'] = df_lags.groupby('product_id')['tn'].shift(lag)
    return df_lags

df_lags = crear_lags(df_prod)

# 4. Agregar variable mes
df_lags['mes'] = df_lags['periodo'].dt.month

# 5. Eliminar filas con NaN por los lags
df_train = df_lags.dropna()

# 6. Separar features y target
features = ['lag_1', 'lag_2', 'lag_3', 'mes']
target = 'tn'

# 7. Separar train y validación
train_data = df_train[df_train['periodo'] < '2019-12-01']
val_data = df_train[df_train['periodo'] == '2019-12-01']

X_train = train_data[features]
y_train = train_data[target]
X_val = val_data[features]
y_val = val_data[target]

# 8. Hiperparámetros de Optuna
mejores_params = {
    'n_estimators': 480,
    'learning_rate': 0.03970088518771,
    'max_depth': 7,
    'num_leaves': 119,
    'subsample': 0.9,
    'colsample_bytree': 0.75,
    'random_state': 42,
    'min_data_in_leaf': 43,
    'reg_alpha': 1.3007611422443943,  # lambda_l1
    'reg_lambda': 3.395240691772275   # lambda_l2
}

modelo = lgb.LGBMRegressor(**mejores_params)
modelo.fit(X_train, y_train)

# 9. Validación
y_pred_val = modelo.predict(X_val)
print("📊 MAE en diciembre 2019:", mean_absolute_error(y_val, y_pred_val))

# 10. Predicción enero 2020
enero_df = df_prod[df_prod['periodo'] == '2019-12-01'].copy()
enero_df['periodo'] = pd.to_datetime('2020-01-01')
enero_df['lag_1'] = df_prod[df_prod['periodo'] == '2019-12-01']['tn'].values
enero_df['lag_2'] = df_prod[df_prod['periodo'] == '2019-11-01']['tn'].values
enero_df['lag_3'] = df_prod[df_prod['periodo'] == '2019-10-01']['tn'].values
enero_df['mes'] = 1
enero_df['tn'] = modelo.predict(enero_df[features])

# 11. Predicción febrero 2020
feb_df = pd.DataFrame()
feb_df['product_id'] = enero_df['product_id']
feb_df['periodo'] = pd.to_datetime('2020-02-01')
feb_df['lag_1'] = enero_df['tn']  # predicción enero
feb_df['lag_2'] = df_prod[df_prod['periodo'] == '2019-12-01']['tn'].values
feb_df['lag_3'] = df_prod[df_prod['periodo'] == '2019-11-01']['tn'].values
feb_df['mes'] = 2
feb_df['tn_predicho'] = modelo.predict(feb_df[features])

# 12. Exportar resultados
resultado = feb_df[['product_id', 'tn_predicho']]
resultado.to_csv("prediccion_feb2020_lightgbm.csv", index=False)
print("✅ Predicción de febrero 2020 guardada en prediccion_feb2020_lightgbm.csv")


/home/waldo/miniconda3/envs/lightgbm_env/lib/python3.10/site-packages/lightgbm/basic.py:18: UserWarning: A NumPy version >=1.22.4 and <1.29.0 is required for this version of SciPy (detected version 1.22.3)
  import scipy.sparse


[LightGBM] [Warning] min_data_in_leaf is set=43, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=43
📊 MAE en diciembre 2019: 8.40573661494098
✅ Predicción de febrero 2020 guardada en prediccion_feb2020_lightgbm.csv
